# Técnicas de Inteligencia Artificial - Introducción a Arboles de Decisión con Python

Para poder trabajar con Python de forma rápida, sencilla y desde cualquier computadora vamos a utilizar este entorno de programación llamado Google Colaboratory o, simplemente Colab, el cual es un servicio que forma parte de los servicios Cloud de Google. 
Colab está basado en los entornos Jupyter Notebooks, los cuales nos permiten escribir y ejecutar código en Python en diferentes celdas sin un orden fijo, al igual que Matlab. Además, los Notebooks de Jupyter permiten intercalar celdas de texto (como esta que estas leyendo) donde se puede complementar con información, agregar fórmulas mediante Latex, insertar imágenes o gráficas, entre otras. 

Para trabajar con árboles de decisión en Python vamos a utilizar la librería [SciKit-Learn](https://scikit-learn.org/stable/index.html). Esta librería implementa una gran variedad de algoritmos de aprendizaje automatizado junto con herramientas para su entrenamiento, refinamiento y validación; conjuntos de datos y algoritmos para su pre-procesamiento, entre otras. SciKit-Learn es ampliamente utilizada en ambientes científicos y de investigación, así como también en la industria principalmente debido a su potencia y simplicidad.

Esta librería, al igual que muchas otras, ya se encuentran instaladas por defecto en Colab.

---


## Preparación del conjunto de datos

Como se mencionó anteriormente, SciKit-Learn trae incorporados varios conjuntos de datos comunmente utilizados en problemas básicos de aprendizaje automatizado. Todos ellos se encuentran en el módulo [`datasets`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) de la librería. A continuación importamos la función [`load_iris`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris) que permite cargar el dataset [Iris](https://archive.ics.uci.edu/ml/datasets/iris) e imprimimos algunos datos sobre el mismo.

Este dataset consiste de datos correspondientes a 3 variedades de flor de Iris. Cada elemento del dataset esta representado por 4 atributos (ancho y largo del pétalo y del sépalo de la flor). En total el dataset consta de 50 ejemplos de cada una de las clases.

In [ ]:
from sklearn.datasets import load_iris
 
dataset = load_iris()

# Imprimo informacion para analizar el conjunto de datos
 
print("Los atributos de entrada son: {}.".format(dataset.feature_names))
print("Las clases que intentaremos predecir son: {}.".format(dataset.target_names))
print("El formato de la matriz de datos es: {}.".format(dataset.data.shape))
print("El formato de la matriz de eitquetas es: {}.".format(dataset.target.shape))

## Conjunto de entrenamiento y evaluación

Una vez cargado el dataset, podemos generar subconjuntos del mismo para el entrenamiento y evaluación del modelo. 

Para ello, hacemos uso de la función [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) del módulo [`model_selection`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection).


In [ ]:
# Divido los datos en conjunto de entrenamiento y evaluacion
from sklearn.model_selection import train_test_split
 
data_train, data_test, target_train, target_test = train_test_split(dataset.data,
                                                                    dataset.target,
                                                                    test_size = 0.2)
 
print("Ahora, el conjunto de entrenamiento tiene {} muestras y el de evaluación tiene {} muestras.".format(data_train.shape[0], data_test.shape[0]))

## Arbol de Decisión

Para crear, entrenar y evaluar un árbol de decisión utilizamos la clase [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) del módulo [`tree`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier). Al momento de crear el árbol, la función nos permite configurar distintos parámetros, entre los que se encuentran:

*  `criterion`: Cadena de texto que puede tomar dos valores: `"gini"` o `"entropy"`. Este parámetro establece el criterio de optimización para determinar cual sera el atributo a utilizar para dividir un nodo y su valor de corte.
*  `max_depth`: Número entero indicando la profundidad máxima que puede adoptar el árbol. Si se lo deja en `None` el árbol se expandira hasta que todas las hojas sean puras, o hasta que todas las hojas contengan menos muestras que `min_samples_leaf`.
*  `min_samples_split`: Número entero indicando la cantidad mínima de muestras necesaria para dividir un nodo en dos nuevos nodos y/o hojas.
*  `min_samples_leaf`: Número entero indicando la cantidad mínima de ejemplos necesaria para formar un nodo hoja. Se va a considerar la separación de un nodo en hojas solo si quedan, al menos `min_samples_leaf` ejemplos de entrenamiento, en cada una de las ramas que se derivan.
*  `max_leaf_nodes`: Número entero indicando la cantidad máxima de hojas que puede tener el árbol.

Mediante estos parámetros somos capaces de controlar las reglas de parada en el entrenamiento del modelo, con el fin de evitar el sobreentrenamiento del mismo. 

In [ ]:
# Definicion de parametros para el entrenamiento del arbol de decision
criterion = 'gini'
max_depth = None
min_samples_split = 2
min_samples_leaf = 1
max_leaf_nodes = None

In [ ]:
# Creamos el modelo y lo entrenamos
from sklearn.tree import DecisionTreeClassifier
 
tree_model = DecisionTreeClassifier(criterion = criterion,
                                    splitter = "best",
                                    max_depth = max_depth,
                                    min_samples_leaf = min_samples_leaf,
                                    min_samples_split = min_samples_split,
                                    max_leaf_nodes = max_leaf_nodes)
 
# Utilizamos el conjunto de datos de entrenamiento
 
tree_model.fit(data_train, target_train)

## Graficar la estructura del árbol

Para poder graficar el árbol obtenido, su estructura de ramas y hojas, y algunos valores obtenidos luego del entrenamiento, tenemos dos opciones:

1.   Utilizar la función [`plot_tree`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html#sklearn.tree.plot_tree) del mismo modulo `tree`.
2.   Utilizar la función `graph_tree` implementada a continuación.



In [ ]:
from sklearn.tree import plot_tree
 
_ = plot_tree(tree_model, feature_names=dataset.feature_names, class_names=dataset.target_names)

In [ ]:
import graphviz
from sklearn import tree
 
# Funcion para generar en gráfico del arbol.
 
# NOTA: Para reutilizar esta funcion en otro Notebook hay que importar los mismos paquetes que se
# importan en esta celda.
def graph_tree(tree_model, feature_names=None, class_names=None):
 
  dot_data = tree.export_graphviz(tree_model, out_file=None,
                                  feature_names=feature_names,
                                  class_names=class_names,
                                  filled=True,
                                  rounded=True,
                                  special_characters=True)  
 
  graph = graphviz.Source(dot_data)
  return graph

In [ ]:
graph = graph_tree(tree_model, dataset.feature_names, dataset.target_names)
graph

## Evaluación
Para evaluar la performance del modelo entrenado sobre los datos vamos a utilizar la métrica de accuracy, definida por la siguiente expresión:

$accuracy(y,\hat y)=\frac {1}{n_{samples}} \displaystyle\sum_{i=0}^{n_{samples}-1}1(\hat y_i=y_i)$

Donde $\hat y_i$ es el valor predicho en la $i$_esima muestra y $y_i$ es el valor correcto que se debe predecir.

Podemos calcular esta métrica mediante la función [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score) del módulo [`metrics`](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics). Para ello haremos una predicción, con el modelo ya entrenado, sobre los datos del conjunto de evaluación.

In [ ]:
from sklearn.metrics import accuracy_score

# Utilizo el conjunto de evaluación data_test para predecir mediante el arbol ya entrenado
target_predicted = tree_model.predict(data_test)

# Calculo el valor de accuracy obtenido
accuracy = accuracy_score(target_test, target_predicted)

print("El valor de accuracy obtenido es: {}".format(accuracy))

Pero, ¿Qué pasa si evalúo el mismo árbol sobre el conjunto de entrenamiento?...

## Sobreentrenamiento

El sobreentrenamiento (overfitting) es el mayor obstáculo a la hora de entrenar un árbol de decisión, y casi cualquier modelo en aprendizaje automatizado. Si los hiperparametros no se ajustan correctamente el árbol crecerá excesivamente, lo cual puede resultar en un 100% de precisión sobre el conjunto de datos de entrenamiento, siendo el peor de los casos aquel en el que cada observación de dicho conjunto genere una hoja propia.

In [ ]:
# Chequeamos si el árbol esta sobreentrenado evaluandolo con algunos datos del conjunto de entrenamiento

target_predicted = tree_model.predict(data_train)

train_accuracy = accuracy_score(target_train, target_predicted)

print("El valor de accuracy obtenido es: {}".format(train_accuracy))

### Ajustar las restricciones (pre-prunning)

Una de las formas en las que se puede evitar el sobreentrenamiento de estos modelos es mediante el ajuste minusioso de sus hiperparametros, de forma tal que limiten el crecimiento del árbol y, por lo tanto, su sobreentrenamiento. 

Algunos ejemplos de limitaciones podrían ser:

*   Reducir la profundidad máxima que puede alcanzar.
*   Aumentar la cantidad de observaciones necesarias para que un nodo se pueda considerar como hoja.
*   Reducir la cantidad máxima de hojas que puede generar el árbol
*   Aumentar la cantidad de observaciones necesarias para que un nodo pueda separarse en dos nuevos nodos/hojas.


### Poda (post-prunning)

La forma mas comun y efectiva de combatir el sobreentrenamiento en los árboles de decisión es la poda. El proceso de poda de un árbol de decisión consiste en eliminar subsecciones del mismo, transformandolas en nodos hojas que representen la clase mas común de los ejemplos de entrenamiento mas utilizados en esa subsección. Se considera, entonces, que las subsecciones eliminadas no representaban información critica, por lo que no permitian la generalización del conocimiento por parte del modelo.

A continuación se implementa una función de poda, cuyo funcionamiento es similar a la de Matlab. A esta función se le pasa como argumento el modelo de árbol que se desea podar y la cantidad de niveles que se le quieren sacar y devolverá el árbol podado.



In [ ]:
from sklearn.tree._tree import TREE_LEAF
from copy import deepcopy

# Funcion para podar un árbol.

# NOTA: Para reutilizar esta función en otra Notebook hay que importar los mismos paquetes que se
# importan en esta celda.

def is_leaf(tree_model, node_id):
  """
  Devuelve True si el nodo (node_id), pasado como
  argumento, es un nodo hoja del arbol (tree_model).
  Caso contrario retorna False.
  """
  return (tree_model.tree_.children_left[node_id] == TREE_LEAF and 
          tree_model.tree_.children_right[node_id] == TREE_LEAF)


def prune(tree_model, levels=1):
  """
  Realiza la poda del arbol pasado como argumento, de forma recursiva, eliminando niveles del mismo.

  Esta funcion replica el funcionamiento de su correspondiente par en Matlab:

          prune(tree_model, 'level', levels)
  """
  tree_model_copy = deepcopy(tree_model)

  def recursive(tree_model, node_id):

    if (is_leaf(tree_model, tree_model.tree_.children_left[node_id]) and
        is_leaf(tree_model, tree_model.tree_.children_right[node_id])):
      tree_model.tree_.children_left[node_id] = TREE_LEAF
      tree_model.tree_.children_right[node_id] = TREE_LEAF
    
    if tree_model.tree_.children_left[node_id] != TREE_LEAF:
      recursive(tree_model, tree_model.tree_.children_left[node_id])
      recursive(tree_model, tree_model.tree_.children_right[node_id])
      
    return tree_model
  
  for _ in range(levels):
    tree_model_copy = recursive(tree_model_copy, 0)

  return tree_model_copy

In [ ]:
# Podo el árbol y lo grafico nuevamente
pruned_tree_model = prune(tree_model, 2)

graph = graph_tree(pruned_tree_model, dataset.feature_names, dataset.target_names)
graph

Una vez podado el árbol, reevaluo con ambos conjuntos de datos para corrobar si el sobreentrenamiento se corregido.

In [ ]:
# Chequeamos si el árbol esta sobreentrenado evaluandolo con algunos datos del conjunto de entrenamiento

target_predicted = pruned_tree_model.predict(data_test)

test_accuracy = accuracy_score(target_test, target_predicted)

print("El valor de accuracy obtenido en el conjunto de evaluacion es: {}".format(test_accuracy))

target_predicted = pruned_tree_model.predict(data_train)

train_accuracy = accuracy_score(target_train, target_predicted)

print("El valor de accuracy obtenido en el conjunto de entrenamiento es: {}".format(train_accuracy))

## Análisis de Hiperparametros

Una forma simple pero efectiva de analizar como se comporta un modelo ante distintos valores de un cierto hiperparametro consiste en entrenar varias veces el mismo modelo, variando el hiperparametro que se desea analizar, y graficando el comportamiento el modelo luego de cada entrenamiento, en funcion del valor del hiperparametro utilizado. 

En la siguiente celda de código se entranará un árbol de decisíon variando el valor de `min_samples_split` desde 2 hasta 10, y se graficará el valor de accuracy obtenido sobre el conjunto de testeo, en función de los asignados a `min_samples_split`.

Para poder generar una gráfica utilizamos la función [`plot`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) de la librería [matplotlib](https://matplotlib.org/).

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from matplotlib.pyplot import plot, show


min_samples_split_values = list(range(2, 10))
accuracy_values = []

for value in min_samples_split_values:

  tree_model = DecisionTreeClassifier(min_samples_split = value)
  tree_model.fit(data_train, target_train)
  predicted_values = tree_model.predict(data_test)
  accuracy = accuracy_score(target_test, predicted_values)

  accuracy_values.append(accuracy)


plot(min_samples_split_values, accuracy_values)
show()


## Validación Cruzada

La validación cruzada (o cross validation) es una técnica ampliamente utilizada en el entrenamiento de modelos de aprendizaje automatizado, que sirve para evaluar la performance de dicho modelo. La forma mas utilizada de validacion cruzada es la de tipo K-Fold. La misma consiste en subdivir el conjunto de entrenamiento en K particiones y repetir el proceso de entrenamiento y evaluación K veces, dejando siempre una partición distinta para la evaluación y entrenando con el resto. El valor final de precisión del modelo será, entonces, el promedio de los valores de precisión obtenidos en cada entrenamiento.

![grid_search_cross_validation.png](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

Para poder implementar validación cruzada de tipo K-Fold con SciKit-Learn debemos utilizar la función [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) del modulo [`model_selection`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection).

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

K = 10

tree_model = DecisionTreeClassifier()

score = cross_validate(tree_model, dataset.data, dataset.target, cv = K)

print("El valor de accuracy obtenido en el conjunto de datos es: {}".format(score["test_score"].mean()))